<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher_New/blob/main/Making_Mirror_Image_and_Fusion_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#左右の画像の鏡像を作成、反対側の画像と合算する。
これまでの実験によって、両手のX線画像を解析するにあたり、モデルには左右の手を正しく区別して認識することは困難であるということが判明した。これを受けて、片手ずつ段階的に訓練を施して、最終的に両手を同時に正しく認識できるようにするほうが現実的であるという結論に達した。このため、まず左手と右手を認識する簡単なbounding_boxを描画し、その境界線に沿って画像をクロップ、右手と左手のX線画像を分割することに成功した。
手指は左右対称であるため、同時に解析するのでなければ、左右反転した左手の画像をも右手の画像として扱えば、倍のサンプル数で訓練が可能になる。そこで本日は、'xpFiles_right'および'xpFiles_left'の鏡像を作成し、反対側の手の画像ファイルと融合させた、'xpFilesRightHand'と'xpFilesLeftHand'を作成し、Macにダウンロードする。

In [1]:
# GoogleDriveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 必要なライブラリのインポート
import os
from PIL import Image

In [3]:
# 作業開始前にBack_upファイルを作成する。
import shutil
shutil.copytree('/content/drive/MyDrive/xpFiles_right','/content/drive/MyDrive/xpFiles_right_back_up')
shutil.copytree('/content/drive/MyDrive/xpFiles_left','/content/drive/MyDrive/xpFiles_left_back_up')

'/content/drive/MyDrive/xpFiles_left_back_up'

In [4]:
# ファイルパスの設定
right_hand_dir = '/content/drive/MyDrive/xpFiles_right'
left_hand_dir = '/content/drive/MyDrive/xpFiles_left'

# 鏡像ファイルのディレクトリを作成（存在しない場合）
right_hand_mirror_dir = '/content/drive/MyDrive/xpFiles_right_mirror'
left_hand_mirror_dir = '/content/drive/MyDrive/xpFiles_left_mirror'
os.makedirs(right_hand_mirror_dir, exist_ok=True)
os.makedirs(left_hand_mirror_dir, exist_ok=True)

In [5]:
# 鏡像変換関数
def mirror_images(input_dir, output_dir):
    for image_name in os.listdir(input_dir):
        if image_name.endswith('.png'):  # PNGファイルのみ処理
            image_path = os.path.join(input_dir, image_name)
            with Image.open(image_path) as img:
                mirrored_img = img.transpose(Image.FLIP_LEFT_RIGHT)
                mirrored_img.save(os.path.join(output_dir, image_name))

# 鏡像変換の実行
mirror_images(right_hand_dir, right_hand_mirror_dir)
mirror_images(left_hand_dir, left_hand_mirror_dir)

In [6]:
# 結合ファイルの作成
def combine_images(dir1, dir2, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    combined_images = {img_name for img_name in os.listdir(dir1) if img_name.endswith('.png')}
    combined_images.update(img_name for img_name in os.listdir(dir2) if img_name.endswith('.png'))

    for img_name in combined_images:
        img_path1 = os.path.join(dir1, img_name)
        img_path2 = os.path.join(dir2, img_name)
        img1_exists = os.path.exists(img_path1)
        img2_exists = os.path.exists(img_path2)

        if img1_exists:
            with Image.open(img_path1) as img1:
                img1.save(os.path.join(output_dir, img_name))
        if img2_exists:
            with Image.open(img_path2) as img2:
                img2.save(os.path.join(output_dir, f'mirrored_{img_name}'))

# xpFilesRightHand と xpFilesLeftHand の作成
combine_images(right_hand_dir, left_hand_mirror_dir, '/content/drive/MyDrive/xpFilesRightHand')
combine_images(left_hand_dir, right_hand_mirror_dir, '/content/drive/MyDrive/xpFilesLeftHand')